In [5]:
import os
import json
import pandas as pd

In [6]:
data_path = os.path.join('..', 'data' ,'raw', 'train.json')
f = open(data_path)
data = json.load(f)
f.close()

In [7]:
data_df = pd.DataFrame(data)
data_df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [8]:
all_labels = set([label for sublist in data_df['labels'] for label in sublist])
all_labels
label_counts = (data_df['labels'].apply(pd.Series).stack().value_counts())
label_counts

O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [9]:
import re 

EMAIL_REGEX = r'\b[A-Za-z0-9]+[.-_]*[A-Za-z0-9]+@(?:gmail|hotmail|yahoo)\.[A-Z|a-z]{2,}\b'
PHONE_NUMEBR_WITH_DOT_REGEX = r'\b\d{3}\.\d{1,4}\.\d{1,10}\b'
PHONE_SECOND_PART_REGEX = r'\b\d{1,3}\)\d{1,4}\b'
PHONE_FOURTH_PART_REGEX =  r'\b\d+(?:x\d+)?\b'
NUMBER_REGEX = r'\d+'
ZIP_CODE_REGEX = r'\b[A-Z]{2}\b'

def check_email(text):
    return re.findall(re.compile(EMAIL_REGEX), text)

def check_phone_with_dot(text):
    return re.findall(re.compile(PHONE_NUMEBR_WITH_DOT_REGEX), text)

def check_phone_second_part(token):
    return re.fullmatch(re.compile(PHONE_SECOND_PART_REGEX), token)

def check_phone_fourth_part(token):
    return re.fullmatch(re.compile(PHONE_FOURTH_PART_REGEX), token)

def check_zip_code(token):
    return re.fullmatch(re.compile(ZIP_CODE_REGEX), token)

def check_number(token):
    return re.fullmatch(re.compile(NUMBER_REGEX), token)


Filter tokens whose tag contains STREET_ADDRESS

In [10]:
addresses = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if '-STREET_ADDRESS' in label]
addresses

[('591', 'B-STREET_ADDRESS'),
 ('Smith', 'I-STREET_ADDRESS'),
 ('Centers', 'I-STREET_ADDRESS'),
 ('Apt', 'I-STREET_ADDRESS'),
 ('.', 'I-STREET_ADDRESS'),
 ('656', 'I-STREET_ADDRESS'),
 ('\n', 'I-STREET_ADDRESS'),
 ('Joshuamouth', 'I-STREET_ADDRESS'),
 (',', 'I-STREET_ADDRESS'),
 ('RI', 'I-STREET_ADDRESS'),
 ('95963', 'I-STREET_ADDRESS'),
 ('743', 'B-STREET_ADDRESS'),
 ('Erika', 'I-STREET_ADDRESS'),
 ('Bypass', 'I-STREET_ADDRESS'),
 ('Apt', 'I-STREET_ADDRESS'),
 ('.', 'I-STREET_ADDRESS'),
 ('419', 'I-STREET_ADDRESS'),
 ('\n', 'I-STREET_ADDRESS'),
 ('Andreahaven', 'I-STREET_ADDRESS'),
 (',', 'I-STREET_ADDRESS'),
 ('IL', 'I-STREET_ADDRESS'),
 ('54207', 'I-STREET_ADDRESS')]

Filter tokens whose tag is B-EMAIL

In [11]:
emails = [token for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if label == 'B-EMAIL']
emails

['djones@gmail.com',
 'matthew72@hotmail.com',
 'belindarojas@yahoo.com',
 'kennethevans@hotmail.com',
 'agood@gmail.com',
 'agood@gmail.com',
 'hwillis@gmail.com',
 'kellyharrison@gmail.com',
 'kellyharrison@gmail.com',
 'lowetyler@hotmail.com',
 'murraythomas@gmail.com',
 'vmartinez@hotmail.com',
 'sheena82@yahoo.com',
 'nbarker@hotmail.com',
 'nbarker@hotmail.com',
 'nbarker@hotmail.com',
 'johnsondavid@hotmail.com',
 'johnsondavid@hotmail.com',
 'catherine19@hotmail.com',
 'john03@yahoo.com',
 'john03@yahoo.com',
 'lisarose@gmail.com',
 'diazkristen@gmail.com',
 'lisarose@gmail.com',
 'diazkristen@gmail.com',
 'andrew58@gmail.com',
 'boydcynthia@yahoo.com',
 'laurie33@yahoo.com',
 'laurie33@yahoo.com',
 'pauljohnson@hotmail.com',
 'pauljohnson@hotmail.com',
 'jarviscindy@hotmail.com',
 'jarviscindy@hotmail.com',
 'jray@hotmail.com',
 'gday@hotmail.com',
 'gday@hotmail.com',
 'gday@hotmail.com',
 'noah14@hotmail.com',
 'hbrown@yahoo.com']

Filter tokens whose tag contains PHONE_NUM

In [12]:
phones = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if '-PHONE_NUM' in label]
phones

[('(', 'B-PHONE_NUM'),
 ('320)202', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('0688x95843', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('223)392', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('2765', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('410.526.1667', 'B-PHONE_NUM')]

Check '(' token label

In [13]:
paranthesis_tokens = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if token=='(']
paranthesis_labels = {item for sublist in paranthesis_tokens for item in sublist if item != '('}
paranthesis_labels

{'B-PHONE_NUM', 'O'}

Chech '-' token label

In [14]:
hyphen_tokens = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if token=='-']
hyphen_labels = {item for sublist in hyphen_tokens for item in sublist if item != '-'}
hyphen_labels

{'I-PHONE_NUM', 'O'}

Filter number tokens with their labels

In [15]:
numbers = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if check_number(token)]
n  = {item for sublist in numbers for item in sublist if not check_number(item)}
n

{'B-ID_NUM',
 'B-STREET_ADDRESS',
 'I-ID_NUM',
 'I-PHONE_NUM',
 'I-STREET_ADDRESS',
 'O'}

In [79]:
def predict_label(row):
    tokens = row['tokens']
    labels = ['O']*len(tokens)
    
    for index, token in enumerate(tokens):
        
        if check_email(token):
            labels[index] = 'B-EMAIL'
        
        if check_phone_with_dot(token):
            labels[index] = 'B-PHONE_NUM'            
        
        try:
            # check phone numbers in the '(320)202-0688x95843' format
            if token == "(":
                second_token = tokens[index+1]
                if check_phone_second_part(second_token):
                    third_token = tokens[index+2]
                    if third_token == "-":
                        fourth_token=tokens[index+3]
                        if check_phone_fourth_part(fourth_token):
                            labels[index] = "B-PHONE_NUM"
                            labels[index+1] = "I-PHONE_NUM"
                            labels[index+2] = "I-PHONE_NUM"
                            labels[index+3] = "I-PHONE_NUM" 
            
            # check address
            # address format: begin with numbers, end with numbers preceded by zip code preceded by comma
            # second condition for when there are numbers within the address
            #todo: default address is considered to be 11, need to investigate more
            if check_number(token) and labels[index]=='O': 
                pointer = index
                match = False
                while not match:
                    pointer += 1
                    if check_zip_code(tokens[pointer]):
                        if tokens[pointer-1] == ",":
                            pointer += 1
                            if check_number(tokens[pointer]):
                                if pointer - index == 10:
                                    match = True
                                    labels[index] = 'B-STREET_ADDRESS'
                                    while pointer != index:
                                        labels[pointer] = 'I-STREET_ADDRESS'
                                        pointer -= 1

        except IndexError:
            pass
    
    return labels

In [75]:
data_df['custom_labels'] = data_df.apply(predict_label, axis=1)
data_df

,document,full_text,tokens,trailing_whitespace,labels,custom_labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


Check the custom label with real label

In [78]:
real_emails = [token for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if label == 'B-EMAIL']
predicted_emails = [token for tokens, labels in zip(data_df['tokens'], data_df['custom_labels']) for token, label in zip(tokens, labels) if label == 'B-EMAIL']

real_phones = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if '-PHONE_NUM' in label]
predicted_phones = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['custom_labels']) for token, label in zip(tokens, labels) if '-PHONE_NUM' in label]

real_address = 
predicted_address = 


True